In [1]:
import numpy as np
import pandas as pd

In [3]:
train=pd.read_csv("../data/raw/train.csv")
store=pd.read_csv("../data/raw/store.csv")
df=pd.merge(train, store, on="Store", how="left")
df.head()

C:\Users\Rohith Kumar\AppData\Local\Temp\ipykernel_18980\547468153.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train=pd.read_csv("../data/raw/train.csv")


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
df=df[df['Open']==1]
df=df[df['Sales']>1]

In [6]:
df['Date']=pd.to_datetime(df['Date'])
df=df.reset_index(drop=True)
df.shape

(844338, 18)

In [7]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekOfYear'] = df['Date'].dt.isocalendar().week.astype(int)


In [8]:
# Weekend / Weekday
df['IsWeekend'] = df['DayOfWeek'].isin([5,6]).astype(int)

# Quarter
df['Quarter'] = df['Date'].dt.quarter


In [9]:
df['Promo'] = df['Promo'].astype(int)

# Long-term promo indicator (from dataset)
df['Promo2'] = df['Promo2'].fillna(0).astype(int)


In [10]:
df['CompetitionDistance'] = df['CompetitionDistance'].fillna(df['CompetitionDistance'].median())

# Flag for competition present
df['HasCompetition'] = (df['CompetitionDistance'] < 5000).astype(int)


In [11]:
# Sales per customer (rough demand indicator)
df['SalesPerCustomer'] = df['Sales'] / df['Customers']


In [12]:
df = df.sort_values(by=['Store','Date'])

df['Sales_Lag_1'] = df.groupby('Store')['Sales'].shift(1)
df['Sales_Lag_7'] = df.groupby('Store')['Sales'].shift(7)
df['Sales_Lag_30'] = df.groupby('Store')['Sales'].shift(30)


In [13]:
df['Sales_Rolling_7'] = df.groupby('Store')['Sales'].shift(1).rolling(7).mean()
df['Sales_Rolling_30'] = df.groupby('Store')['Sales'].shift(1).rolling(30).mean()


In [14]:
df = df.dropna().reset_index(drop=True)
df.shape


(256838, 31)

In [16]:
feature_cols = [
    'Store',
    'Promo', 'Promo2',
    'CompetitionDistance', 'HasCompetition',
    'DayOfWeek', 'IsWeekend', 'WeekOfYear', 'Quarter',
    'Sales_Lag_1', 'Sales_Lag_7', 'Sales_Lag_30',
    'Sales_Rolling_7', 'Sales_Rolling_30'
]

target_col = 'Sales'

X = df[feature_cols]
y = df[target_col]



In [17]:

X.head(), y.head()

(   Store  Promo  Promo2  CompetitionDistance  HasCompetition  DayOfWeek  \
 0      2      1       1                570.0               1          2   
 1      2      1       1                570.0               1          3   
 2      2      1       1                570.0               1          4   
 3      2      0       1                570.0               1          5   
 4      2      0       1                570.0               1          0   
 
    IsWeekend  WeekOfYear  Quarter  Sales_Lag_1  Sales_Lag_7  Sales_Lag_30  \
 0          0           6        1       6269.0       3461.0        4422.0   
 1          0           6        1       6438.0       4446.0        4159.0   
 2          0           6        1       5575.0       4340.0        4484.0   
 3          1           6        1       4203.0       4545.0        2342.0   
 4          0           7        1       3031.0       2956.0        6775.0   
 
    Sales_Rolling_7  Sales_Rolling_30  
 0      4642.428571       4514.0

In [18]:
df_final = df[feature_cols + ['Sales']]
df_final.to_csv("../data/processed/model_input.csv", index=False)

print("Saved → data/processed/model_input.csv")


Saved → data/processed/model_input.csv
